In [1]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


# Data 불러오기

In [17]:
mbti_all_title = pd.read_csv('/opt/ml/input/final_project/data/mbti/mbti_all_title.csv')
mbti_all = pd.read_csv('/opt/ml/input/final_project/data/Personality_All_168000_rows.csv')

In [2]:
data = pd.read_csv('/opt/ml/input/final_project/Crawling/imdb_side_df_EmptyList_to_None.csv') # side information data
data = data.drop_duplicates('Contents',keep='first') # 중복 제거
data.drop(['produceers', 'composers'], axis=1, inplace=True) # 수집 하나도 안되서 쓸모 없음
data

,Contents,searched_content,genres,plot,directors,actors
0,Moana (2016),Moana,"['Animation', 'Adventure', 'Comedy', 'Family',...","[""In Ancient Polynesia, when a terrible curse ...","['Ron Clements', 'John Musker', 'Don Hall', 'C...","[""Auli'i Cravalho"", 'Dwayne Johnson', 'Rachel ..."
1,Inside Out (2015),Inside Out,"['Animation', 'Adventure', 'Comedy', 'Drama', ...",['After young Riley is uprooted from her Midwe...,"['Pete Docter', 'Ronnie Del Carmen']","['Amy Poehler', 'Phyllis Smith', 'Richard Kind..."
2,Zootopia (2016),Zootopia,"['Animation', 'Adventure', 'Comedy', 'Crime', ...","['In a city of anthropomorphic animals, a rook...","['Byron Howard', 'Rich Moore', 'Jared Bush']","['Ginnifer Goodwin', 'Jason Bateman', 'Idris E..."
3,Legally Blonde (2001),Legally Blonde,"['Comedy', 'Romance']",NaN,['Robert Luketic'],"['Reese Witherspoon', 'Luke Wilson', 'Selma Bl..."
4,Dead Poets Society (1989),Dead Poets Society,"['Comedy', 'Drama']",['Maverick teacher John Keating uses poetry to...,['Peter Weir'],"['Robin Williams', 'Robert Sean Leonard', 'Eth..."
...,...,...,...,...,...,...
4049,Cloak & Dagger (2018),Cloak & Dagger,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",['Two teenagers from very different background...,NaN,"['Olivia Holt', 'Aubrey Joseph', 'Gloria Reube..."
4050,Peacemaker (2022),Peacemaker,"['Action', 'Adventure', 'Comedy', 'Crime', 'Fa...",['Picking up where The Suicide Squad (2021) le...,NaN,"['John Cena', 'Danielle Brooks', 'Freddie Stro..."
4051,Guardians of the Galaxy (2015),Guardians of the Galaxy,"['Action', 'Adventure', 'Comedy', 'Sci-Fi']",['A group of intergalactic criminals must pull...,['James Gunn'],"['Chris Pratt', 'Zoe Saldana', 'Dave Bautista'..."
4052,Icon Comics,Iron Sky: The Coming Race,"['Action', 'Adventure', 'Comedy', 'Sci-Fi']",['A follow-up to the film Iron Sky (2012) in w...,['Timo Vuorensola'],"['Lara Rossi', 'Vladimir Burlakov', 'Kit Dale'..."


In [3]:
data_no_none_plot = data.loc[data['plot'].isna() == False] # 일단 plot이 비지 않은 데이터만 사용했다

def remove_list_symbol(row):  # DataFrame 만들때 조금 문제가 있었는듯
    return row['plot'][2:-2]  # 리스트 형태가 아니고 str이다 ex "['Batman ...']" 이런식으로 저장되어있어서 ['와 ']를 제거하는 목적

data_no_none_plot['plot'] = data_no_none_plot.apply(remove_list_symbol, axis=1)
data_no_none_plot

/tmp/ipykernel_107915/614043056.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_none_plot['plot'] = data_no_none_plot.apply(remove_list_symbol, axis=1)


,Contents,searched_content,genres,plot,directors,actors
0,Moana (2016),Moana,"['Animation', 'Adventure', 'Comedy', 'Family',...","In Ancient Polynesia, when a terrible curse in...","['Ron Clements', 'John Musker', 'Don Hall', 'C...","[""Auli'i Cravalho"", 'Dwayne Johnson', 'Rachel ..."
1,Inside Out (2015),Inside Out,"['Animation', 'Adventure', 'Comedy', 'Drama', ...",After young Riley is uprooted from her Midwest...,"['Pete Docter', 'Ronnie Del Carmen']","['Amy Poehler', 'Phyllis Smith', 'Richard Kind..."
2,Zootopia (2016),Zootopia,"['Animation', 'Adventure', 'Comedy', 'Crime', ...","In a city of anthropomorphic animals, a rookie...","['Byron Howard', 'Rich Moore', 'Jared Bush']","['Ginnifer Goodwin', 'Jason Bateman', 'Idris E..."
4,Dead Poets Society (1989),Dead Poets Society,"['Comedy', 'Drama']",Maverick teacher John Keating uses poetry to e...,['Peter Weir'],"['Robin Williams', 'Robert Sean Leonard', 'Eth..."
5,Turning Red (2022),Turning Red,"['Animation', 'Adventure', 'Comedy', 'Family',...",A 13-year-old girl named Meilin turns into a g...,['Domee Shi'],"['Rosalie Chiang', 'Sandra Oh', 'Ava Morse', '..."
...,...,...,...,...,...,...
4049,Cloak & Dagger (2018),Cloak & Dagger,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",Two teenagers from very different backgrounds ...,NaN,"['Olivia Holt', 'Aubrey Joseph', 'Gloria Reube..."
4050,Peacemaker (2022),Peacemaker,"['Action', 'Adventure', 'Comedy', 'Crime', 'Fa...",Picking up where The Suicide Squad (2021) left...,NaN,"['John Cena', 'Danielle Brooks', 'Freddie Stro..."
4051,Guardians of the Galaxy (2015),Guardians of the Galaxy,"['Action', 'Adventure', 'Comedy', 'Sci-Fi']",A group of intergalactic criminals must pull t...,['James Gunn'],"['Chris Pratt', 'Zoe Saldana', 'Dave Bautista'..."
4052,Icon Comics,Iron Sky: The Coming Race,"['Action', 'Adventure', 'Comedy', 'Sci-Fi']",A follow-up to the film Iron Sky (2012) in whi...,['Timo Vuorensola'],"['Lara Rossi', 'Vladimir Burlakov', 'Kit Dale'..."


In [4]:
# index로 접근하기 위한 처리
content2idx = {}
idx2content = {}
for i, c in enumerate(data_no_none_plot['Contents']):
    content2idx[c] = i
    idx2content[i] = c

# TF-IDF

In [5]:
# 불용어: 유의미하지 않은 단어 토큰을 제거
# tfidf = TfidfVectorizer(stop_words='english')
tfidf = TfidfVectorizer() 

# plot에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data_no_none_plot['plot'])
print(tfidf_matrix.shape) # -> (컨텐츠 수, 단어 토큰 수) : 하나의 컨텐츠 당 몇개의 단어 토큰이 생겼는지 알 수 있다

(2982, 36761)


In [6]:
cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_matrix # content 간의 plot cosine 유사도

array([[1.        , 0.09490388, 0.08590283, ..., 0.10038982, 0.14462084,
        0.11774504],
       [0.09490388, 1.        , 0.08971355, ..., 0.08090319, 0.09153035,
        0.11796282],
       [0.08590283, 0.08971355, 1.        , ..., 0.07888464, 0.10036422,
        0.12122678],
       ...,
       [0.10038982, 0.08090319, 0.07888464, ..., 1.        , 0.16208113,
        0.12947596],
       [0.14462084, 0.09153035, 0.10036422, ..., 0.16208113, 1.        ,
        0.16074349],
       [0.11774504, 0.11796282, 0.12122678, ..., 0.12947596, 0.16074349,
        1.        ]])

In [31]:
len(cosine_matrix[0])

2982

In [7]:
cosine_matrix.shape

(2982, 2982)

In [8]:
cosine_matrix[0] # Moana
sim_scores = [(i,c) for i, c in enumerate(cosine_matrix[0]) if i != 0] # 자기 자신을 제외한 영화들의 유사도 및 인덱스를 추출
sim_scores

[(1, 0.09490388111825357),
 (2, 0.0859028309749217),
 (3, 0.08563049872370243),
 (4, 0.07478223350450545),
 (5, 0.08449467266047209),
 (6, 0.09136354258371376),
 (7, 0.11369937542597434),
 (8, 0.09013263554198087),
 (9, 0.05413646182707817),
 (10, 0.09378356348595011),
 (11, 0.05766288443530682),
 (12, 0.09069898599631367),
 (13, 0.086179529782388),
 (14, 0.1284639571659063),
 (15, 0.11424122522153489),
 (16, 0.0987151025937099),
 (17, 0.11384377522823289),
 (18, 0.10161925549697289),
 (19, 0.11184658934124794),
 (20, 0.09198701459883753),
 (21, 0.1067300172657905),
 (22, 0.07439622120438452),
 (23, 0.11552652284698055),
 (24, 0.08932923180211394),
 (25, 0.10642848002795825),
 (26, 0.07587575016707387),
 (27, 0.08520585220155663),
 (28, 0.051013511740197116),
 (29, 0.06144283589838297),
 (30, 0.08052065810342478),
 (31, 0.09102340935872086),
 (32, 0.1243412906383558),
 (33, 0.09696632034902673),
 (34, 0.0925302917485037),
 (35, 0.0804761920615015),
 (36, 0.09096838356250213),
 (37, 0.0

In [9]:
# 유사도가 높은 순서로 정렬
sim_scores = sorted(sim_scores, key= lambda x: x[1],reverse=True)
sim_scores[0:10] # 상위 10개

[(1518, 0.18272851700891274),
 (2707, 0.17884150509147828),
 (2279, 0.16703746689432028),
 (182, 0.1640782572800238),
 (2590, 0.16141721057699088),
 (2222, 0.15960704897026828),
 (1305, 0.15529931516467588),
 (143, 0.15396698360568004),
 (1974, 0.153722944695375),
 (1278, 0.1532142484561895)]

In [10]:
# id를 title로 변환
top10 = [(idx2content[i], score) for i , score in sim_scores[0:10]]
top10

[('Lost', 0.18272851700891274),
 ('Return to the Blue Lagoon (1991)', 0.17884150509147828),
 ('The Blue Lagoon (1980)', 0.16703746689432028),
 ('Paperman (2012)', 0.1640782572800238),
 ('Babel (2006)', 0.16141721057699088),
 ('Das Boot (1981)', 0.15960704897026828),
 ('Cleo from 5 to 7 (1962)', 0.15529931516467588),
 ('Titanic (1997)', 0.15396698360568004),
 ('Elektra (2005)', 0.153722944695375),
 ('M (1931)', 0.1532142484561895)]

In [29]:
top10_movie_and_charactors = {}
for t in top10:
    found_charactors = list(set(mbti_all.loc[mbti_all['Contents'] == t[0]]['Character'].to_list()))
    top10_movie_and_charactors[t[0]] = found_charactors
top10_movie_and_charactors

{'Lost': ['Min jung', 'Peter Mofo', 'Lee Kang Jae', 'Kyung Eun'],
 'Return to the Blue Lagoon (1991)': ['Lilli Hargrave'],
 'The Blue Lagoon (1980)': ['Emmeline Lestrange', 'Richard', 'Paddy Button'],
 'Paperman (2012)': ['George', 'Meg'],
 'Babel (2006)': ['Chieko Wataya'],
 'Das Boot (1981)': ['Kapitenleutnant'],
 'Cleo from 5 to 7 (1962)': ['Cléo', 'Antoine', 'Angèle'],
 'Titanic (1997)': ['Molly Brown',
  'Chief Officer Henry Tingle Wilde',
  'The iceberg',
  'Second Officer Charles Lightoller',
  'Jack Dawson',
  'Captain Edward John Smith',
  'Thomas Andrews',
  'Fabrizio De Rossi',
  'Spicer Lovejoy',
  'Brock Lovett',
  'Tommy Ryan',
  'Ruth DeWitt Bukater',
  'Caledon “Cal” Hockley',
  'Rose Dewitt Bukater',
  'Fifth Officer Harold Lowe'],
 'Elektra (2005)': ['Elektra'],
 'M (1931)': ['Schränker', 'Hans Beckert']}

------------------

## BERT 시도

In [57]:
# Initialize the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize the input sentence
sentence = "Hello, how are you today?"
input_ids = tokenizer.encode(sentence, return_tensors='pt')

# Get the embedding for the input sentence
output = model.forward(input_ids)
embedding = output[0][:,0,:]

print(embedding)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[-5.4739e-02, -7.0617e-02, -3.2456e-01, -7.5682e-02, -1.8231e-01,
         -3.9358e-01,  4.3037e-01,  3.2224e-01,  1.5108e-01, -1.5903e-01,
          1.3377e-01, -3.3188e-01,  7.7551e-02,  2.2687e-01,  1.4729e-01,
         -2.5244e-01, -5.1465e-02,  3.6492e-01,  1.5353e-01,  1.8753e-01,
          1.1702e-02, -4.6375e-01,  3.2672e-01,  1.5534e-01, -7.0134e-02,
         -5.4348e-01, -1.2771e-01,  3.1467e-03, -2.3377e-01, -3.0729e-01,
         -2.4848e-01,  3.4428e-01, -6.2413e-01, -1.9735e-01,  2.5582e-01,
          4.2068e-01,  1.2032e-01,  1.2711e-01, -2.5629e-01,  3.5941e-01,
         -8.5056e-01, -1.0281e-01,  9.7762e-02,  2.9989e-01, -4.2660e-02,
         -9.3860e-01, -3.2108e+00, -2.6338e-01, -3.1378e-01, -3.0535e-01,
          2.8455e-01, -1.5395e-01,  8.2358e-02,  3.3179e-02, -2.8883e-01,
          2.8574e-01, -6.1085e-01, -7.5030e-02, -1.2878e-01,  4.3042e-02,
          1.1005e-01, -1.0142e-01, -4.1042e-01,  2.5719e-01, -1.6000e-01,
          7.0939e-01, -4.9467e-02,  5.

In [51]:
embedding[0]

tensor([-5.4739e-02, -7.0617e-02, -3.2456e-01, -7.5682e-02, -1.8231e-01,
        -3.9358e-01,  4.3037e-01,  3.2224e-01,  1.5108e-01, -1.5903e-01,
         1.3377e-01, -3.3188e-01,  7.7551e-02,  2.2687e-01,  1.4729e-01,
        -2.5244e-01, -5.1465e-02,  3.6492e-01,  1.5353e-01,  1.8753e-01,
         1.1702e-02, -4.6375e-01,  3.2672e-01,  1.5534e-01, -7.0134e-02,
        -5.4348e-01, -1.2771e-01,  3.1467e-03, -2.3377e-01, -3.0729e-01,
        -2.4848e-01,  3.4428e-01, -6.2413e-01, -1.9735e-01,  2.5582e-01,
         4.2068e-01,  1.2032e-01,  1.2711e-01, -2.5629e-01,  3.5941e-01,
        -8.5056e-01, -1.0281e-01,  9.7762e-02,  2.9989e-01, -4.2660e-02,
        -9.3860e-01, -3.2108e+00, -2.6338e-01, -3.1378e-01, -3.0535e-01,
         2.8455e-01, -1.5395e-01,  8.2358e-02,  3.3179e-02, -2.8883e-01,
         2.8574e-01, -6.1085e-01, -7.5030e-02, -1.2878e-01,  4.3042e-02,
         1.1005e-01, -1.0142e-01, -4.1042e-01,  2.5719e-01, -1.6000e-01,
         7.0939e-01, -4.9467e-02,  5.3888e-01, -7.5

In [55]:
# Initialize the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model = model.cuda()

plot_embeddings = []
for p in data_no_none_plot['plot'][:1]:
    # Tokenize the input sentence
    input_ids = tokenizer.encode(p, return_tensors='pt')
    input_ids.cuda()

    token_type_ids = torch.tensor([0] + [1] * (input_ids.shape[1]-1), dtype=torch.long, device=input_ids.device)
    token_type_ids.cuda()

    # Get the embedding for the input sentence
    output = model.forward(input_ids,token_type_ids=token_type_ids)
    embedding = output[0][:,0,:]
    plot_embeddings.append(embedding[0])
